In [8]:
import pandas as pd
from openai import OpenAI
from autoddg import AutoDDG
from autoddg.utils import get_sample

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from autoddg.related.related import RelatedWorkProfiler
# Load environment variables
load_dotenv()

# Your existing MODEL_CONFIG
MODEL_CONFIG = {
    "base_url": "https://openrouter.ai/api/v1",
    "api_key": os.getenv("OPENROUTER_API_KEY"), 
    # "model_name": "mistralai/mistral-7b-instruct:free", 
    "model_name": "google/gemini-2.0-flash-exp:free",
}

# Create client
client = OpenAI(
    api_key=MODEL_CONFIG["api_key"],
    base_url=MODEL_CONFIG["base_url"]
)

# Create profiler
profiler = RelatedWorkProfiler(
    client=client,
    model_name=MODEL_CONFIG["model_name"]
)

In [10]:
auto_ddg = AutoDDG(
    client=client, 
    model_name=MODEL_CONFIG["model_name"]
)

In [11]:
# Test the new analyze_related method
related_profile = auto_ddg.analyze_related(
    pdf_path="../src/autoddg/related/papers/code15.pdf",
    dataset_name="CODE-15%: a large scale annotated dataset of 12-lead ECGs",
    max_pages=10  # Limit pages for quick testing
)

print("="*60)
print("RELATED WORK PROFILE")
print("="*60)
print(related_profile["summary"])

Ignoring wrong pointing object 43 0 (offset 0)


Reading PDF from: ../src/autoddg/related/papers/code15.pdf
Successfully extracted text from 10 pages (total: 10 pages)
Total characters extracted: 55673
Extracting related work profile for dataset: CODE-15%: a large scale annotated dataset of 12-lead ECGs
Sending 56312 characters to LLM...


Exception: Error calling LLM for extraction: Error code: 404 - {'error': {'message': 'No endpoints found for google/gemini-flash-1.5.', 'code': 404}, 'user_id': 'user_3551JjwqBTZXXoc6pSU6LwuJ0ej'}

In [7]:
# Load dataset
df = pd.read_csv("../src/autoddg/related/data/code-15.csv")
sample_df, dataset_sample = get_sample(df, sample_size=100)

# Step 1: Profile the dataset
basic_profile, structural_profile = auto_ddg.profile_dataframe(df)

# Step 2: Analyze semantics
semantic_profile = auto_ddg.analyze_semantics(sample_df)

/opt/homebrew/Caskroom/miniconda/base/envs/nlp_final/lib/python3.12/site-packages/datamart_profiler/core.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.astype(object).fillna('').astype(str)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1764201600000'}, 'provider_name': None}}, 'user_id': 'user_3551JjwqBTZXXoc6pSU6LwuJ0ej'}

In [ ]:
# Step 3: Generate topic
data_topic = auto_ddg.generate_topic("CODE-15%: a large scale annotated dataset of 12-lead ECGs", None, dataset_sample)

# Step 4: Analyze related work (NEW!)
related_profile = auto_ddg.analyze_related(
    pdf_path="../src/autoddg/related/papers/code15.pdf",
    dataset_name="CODE-15%: a large scale annotated dataset of 12-lead ECGs",
    max_pages=10
)

# Step 5: Generate description WITH related work
prompt, description = auto_ddg.describe_dataset(
    dataset_sample=dataset_sample,
    dataset_profile=basic_profile,
    use_profile=True,
    semantic_profile=semantic_profile,
    use_semantic_profile=True,
    data_topic=data_topic,
    use_topic=True,
    related_profile=related_profile,  # Pass the dict here
    use_related_profile=True
)

print(description)

TypeError: AutoDDG.describe_dataset() got an unexpected keyword argument 'related_profile'